In [ ]:
'''
Generate panels for figure 12 

Maximilian Eggl
Nov 2024
'''

import sys
    
from ExtFuncs import *
%load_ext autoreload
%autoreload 2

import scipy.stats as stats

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)

import networkx as nx
import networkx.algorithms.community as nx_comm
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import squareform,pdist

import matplotlib.patches as patches
import matplotlib.colors as mcolors

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

from matplotlib.lines import Line2D

from statsmodels.stats.multitest import multipletests

In [ ]:
datDir = './Dat/'
ColorCode = {'D1Nac':np.array([117/255,20/255,124/255]),'D2Nac':np.array([241/255,164/255,84/255]),
             'mPFC':np.array([86/255,187/255,126/255]),'Sus':np.array((0.9682, 0.7208441, 0.612293)),
            'Res':np.array((0.705673158, 0.01555616, 0.150232812)),'nForg':np.array((0.6672529243333334, 0.779176457, 0.992959213)),
            'Forg':np.array((0.2298057, 0.298717966, 0.753683153))}

def PlotSig(left,right,height,starAdjust=0.02,col = 'k',ax=None, text = None):
    if(ax is None):
        plt.plot([left,right],[height]*2,c=col)
        if(text is None):
            plt.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            plt.text((left+right)/2,height+starAdjust,text,ha='center')
    else:
        ax.plot([left,right],[height]*2,c=col)
        if(text is None):
            ax.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            ax.text((left+right)/2,height+starAdjust,text,ha='center')   


def PlotViolScat(A,col,pos,lab=None,ax= None,hatch=False):
    if(ax is None):
        vp = plt.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        if(hatch):
            vp['bodies'][0].set_hatch('//')
    else:
        vp = ax.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        if(hatch):
            vp['bodies'][0].set_hatch('//')
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        
def GetGraphMetrics(G,labels):
    #degree mean
    mdd = np.mean(sorted((d for n, d in G.degree()), reverse=True))
    #global efficiency
    ge  = nx.global_efficiency(G)
    #betweenness
    be  = np.mean(list(nx.betweenness_centrality(G,normalized=True).values()))
    #modularity
    try: #this messes stuff up - without try gets the figures in E
        t = [np.argwhere(labels==i).squeeze() for i in range(1,max(labels)+1)]
        mod = nx_comm.modularity(G, t)
    except:
        mod = 0
    #assortativity
    ass = nx.degree_pearson_correlation_coefficient(G)
    return mdd,ge,be,mod,ass

In [ ]:
DatA,TimesA,DatNamesA,TimeCalcDatA,CalcDatA,ShockTimesA,cRawA = GetDat('Batch A 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)
DatB,TimesB,DatNamesB,TimeCalcDatB,CalcDatB,ShockTimesB,cRawB = GetDat('Batch B 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)

DatMPC = DatA+DatB
TimesMPC = np.vstack([TimesA,TimesB])
DatNamesMPC = DatNamesA + DatNamesB
TimeCalcDatMPC = TimeCalcDatA+TimeCalcDatB
CalcDatMPC = CalcDatA+CalcDatB
ShockTimesMPC = np.vstack([ShockTimesA,ShockTimesB])
labelsMPC = np.array([1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,rawC = GetDat('Batch C 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,rawD = GetDat('Batch D 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = rawC+rawD
labelsNAD1 = np.array([1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])

In [ ]:
labelsNAD2 = np.array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [ ]:
DatE,TimesE,DatNamesE,TimeCalcDatE,CalcDatE,ShockTimesE,rawE = GetDat('Batch E 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=1)
DatF,TimesF,DatNamesF,TimeCalcDatF,CalcDatF,ShockTimesF,rawF = GetDat('Batch F 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=1)
DatNAD2 = DatE+DatF
TimesNAD2 = np.vstack([TimesE,TimesF])
DatNamesNAD2 = DatNamesE + DatNamesF
TimeCalcDatNAD2 = TimeCalcDatE+TimeCalcDatF
CalcDatNAD2 = CalcDatE+CalcDatF
ShockTimesNAD2 = np.vstack([ShockTimesE,ShockTimesF])
RawDatNAD2 = rawE+rawF
labelsNAD2 = np.array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    TC = TimeCalcDatMPC[j][i]
    ST = ShockTimesMPC[j][i]
    Order = np.zeros(Calc.shape[0])
    Order2 = np.zeros(Calc.shape[0])
    Fire0 = Calc[:,TC<ST[0]].mean(axis=1)==0.0
    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order = np.zeros_like(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    FullOrder.append(Order)
    FullOrderA.append(Order2)
    
print('-'*10 + 'Done 1'+'-'*10)    
i = 1
FullOrder2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)
print('-'*10 + 'Done 2'+'-'*10)
i = 2
FullOrder3 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
print('-'*10 + 'Done 3'+'-'*10)
i = 3
FullOrder4 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
print('-'*10 + 'Done 4'+'-'*10)
TotalOrder = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrder,FullOrder2,FullOrder3,FullOrder4)]

In [ ]:
FullOrderD1 = []
FullOrderD1A = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD1.append(Order)
    FullOrderD1A.append(Order2)

print('1 done')

i = 1
FullOrderD12 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD1[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD1A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD12.append(Order)

print('2 done')

i = 2
FullOrderD13 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD13.append(Order)
    
print('3 done')

i = 3
FullOrderD14 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]<0,FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]>0,FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD14.append(Order)
    
print('4 done')

TotalOrderD1 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD1,FullOrderD12,FullOrderD13,FullOrderD14)]

In [ ]:
FullOrderD2 = []
FullOrderD2A = []
i=0
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    TC = TimeCalcDatNAD2[j][i]
    ST = ShockTimesNAD2[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD2.append(Order)
    FullOrderD2A.append(Order2)

print('1 done')

i = 1
FullOrderD22 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD2[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD2A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD22.append(Order)

print('2 done')

i = 2
FullOrderD23 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD23.append(Order)
    
print('3 done')

i = 3
FullOrderD24 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]<0,FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]>0,FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD24.append(Order)
    
print('4 done')

TotalOrderD2 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD2,FullOrderD22,FullOrderD23,FullOrderD24)]

## Pre-Post

In [ ]:
k = 1
Calc =  CalcDatMPC[k][1]
Calc = Calc[Calc.sum(axis=1)>0]
O = FullOrder2[k]
O = O[~np.isnan(O)]
Calc = Calc[np.argsort(O)]
T = TimesMPC[k][1]
CT = TimeCalcDatMPC[k][1]


CalcPre=Calc[:,CT<T[0]]
Spikes = [SpikeCount(C1) for C1 in CalcPre]

CalcPre = CalcPre[[len(S)>0 for S in Spikes]]

In [ ]:
M = np.zeros((len(CalcPre),len(CalcPre)))
for i,C1 in enumerate(CalcPre):
    for j,C2 in enumerate(CalcPre):
        M[i,j] = np.corrcoef(C1,C2)[0,1]

In [ ]:
Labs =  ['Other','G2','G1',]

In [ ]:
np.fill_diagonal(M,0)
M[abs(M)<0.3] = 0
normalize = mcolors.Normalize(vmin=-0.3, vmax=1)
G = nx.from_numpy_array(M)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
#p = nx.spring_layout(G,k=0.75, iterations=20)
p = nx.circular_layout(G)

fig,ax = plt.subplots()
label_colors = {-2: ColorCode['D1Nac'],-1: ColorCode['D1Nac'], 0: 'gray', 1: ColorCode['D1Nac']+0.3}

node_colors = [label_colors[np.sort(O)[node]] for node in G.nodes()]

nx.draw(G,p,node_color=node_colors,node_size=20, edgelist=edges, edge_color=weights, width=2.0, 
               vmin = -0.3,vmax = 1)
sm = plt.cm.ScalarMappable(
                           norm=plt.Normalize(-0.3,vmax = 1))
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=-0.3, vmax=1))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax,location='left',pad=-0.01)
cbar.set_label('Correlation',fontsize=32,rotation=90)
cbar.set_ticks([-0.2,0,0.2,0.4,0.6,0.8,1.0])
ax.set_aspect('equal')
legend_elements = [Line2D([0], [0], marker='o', color='w', label=Labs[group],
                          markerfacecolor=color, markersize=10) for group, color in label_colors.items()]
ax.legend(handles=legend_elements[1:], ncols=3,
         loc=1,bbox_to_anchor=(1,1.3),fontsize=24,
          columnspacing=0.8,handletextpad=0.1)



In [ ]:
CalcPost=Calc[:,CT>T[0]]
Spikes = [SpikeCount(C1) for C1 in CalcPost]

CalcPost = CalcPost[[len(S)>0 for S in Spikes]]

M = np.zeros((len(CalcPost),len(CalcPost)))
for i,C1 in enumerate(CalcPost):
    for j,C2 in enumerate(CalcPost):
        M[i,j] = np.corrcoef(C1,C2)[0,1]

In [ ]:
np.fill_diagonal(M,0)
M[abs(M)<0.3] = 0
normalize = mcolors.Normalize(vmin=-0.3, vmax=1)
G = nx.from_numpy_array(M)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
#p = nx.spring_layout(G,k=0.75, iterations=20)
p = nx.circular_layout(G)

fig,ax = plt.subplots()
label_colors = {-2: ColorCode['D1Nac'],-1: ColorCode['D1Nac'], 0: 'gray', 1: ColorCode['D1Nac']+0.3}

node_colors = [label_colors[np.sort(O)[node]] for node in G.nodes()]

nx.draw(G,p,node_color=node_colors,node_size=20, edgelist=edges, edge_color=weights, width=2.0, 
               vmin = -0.3,vmax = 1)

ax.set_aspect('equal')

In [ ]:
FigLabs = ['Mean degree \n [edges per node]','Efficiency','Modularity','Betweeness','Assortativity']

In [ ]:
GM = np.zeros((len(CalcDatMPC),4,5))
GM2 = np.zeros((len(CalcDatMPC),4,5))
for i,C1 in enumerate(CalcDatMPC):
    for j,C2 in enumerate(C1[1:]):
        T = TimesMPC[i][j+1]
        CT = TimeCalcDatMPC[i][j+1]
        CalcPre=C2[:,CT<T[0]]
        CalcPre = CalcPre[CalcPre.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPre]
        CalcPre = CalcPre[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPre)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM[i,j,:] = math.nan
            
        CalcPost=C2[:,CT>T[0]]
        CalcPost = CalcPost[CalcPost.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPost]
        CalcPost = CalcPost[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPost)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM2[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM2[i,j,:] = math.nan

In [ ]:
GMwo = np.zeros((len(CalcDatMPC),4,5))
GM2wo = np.zeros((len(CalcDatMPC),4,5))
for i,(C1,TO) in enumerate(zip(CalcDatMPC,TotalOrder)):
    for j,C2 in enumerate(C1[1:]):
        Calc = C2[~TO]
        T = TimesMPC[i][j+1]
        CT = TimeCalcDatMPC[i][j+1]
        CalcPre=Calc[:,CT<T[0]]
        CalcPre = CalcPre[CalcPre.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPre]
        CalcPre = CalcPre[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPre)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GMwo[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GMwo[i,j,:] = math.nan
            
        Calc = C2[~TO]
        CalcPost=Calc[:,CT>T[0]]
        CalcPost = CalcPost[CalcPost.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPost]
        CalcPost = CalcPost[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPost)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM2wo[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM2wo[i,j,:] = math.nan

In [ ]:
i = 1
pval = stats.ttest_ind(GM[:,:-1,i],GM2[:,:-1,i],nan_policy='omit').pvalue
print(pval)
reject_bh, pvals_corrected_bh, _, _ = multipletests(pval, alpha=0.05, method='fdr_bh')
print(pvals_corrected_bh)

In [ ]:
RomNum = ['i','ii','iii','iv','v']
for i in range(5):
    plt.errorbar([1,2,3],np.nanmean(GM[:,:-1,i],axis=0),stats.sem(GM[:,:-1,i],axis=0,nan_policy='omit'),
                 lw=3,label='Pre-start',c=ColorCode['D1Nac'])
    plt.errorbar([1,2,3],np.nanmean(GM2[:,:-1,i],axis=0),stats.sem(GM2[:,:-1,i],axis=0,nan_policy='omit'),
                 lw=3,c=ColorCode['D1Nac'],ls='--',
                 label='Post-start')
    pval_old = stats.ttest_ind(GM[:,:-1,i],GM2[:,:-1,i],nan_policy='omit').pvalue
    x = np.copy(pval_old)
    reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
    x[pval_old<0.05] = pval
    print(x)
    pval = np.copy(x)
    hval = np.max((np.nanmean(GM2[:,:-1,i],axis=0)+stats.sem(GM2[:,:-1,i],axis=0,nan_policy='omit'),
          np.nanmean(GM[:,:-1,i],axis=0)+stats.sem(GM[:,:-1,i],axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:-1,i],axis=0)*0.1
    m1 = (pval<0.05)*(pval>0.01)
    plt.plot(np.array([1,2,3])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    plt.plot(np.array([1,2,3])[m2]-0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m2]+0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    plt.plot(np.array([1,2,3])[m3]-0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m3]+0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.ylabel(FigLabs[i],fontsize=32)
    plt.xticks([1,2,3],['CMR1','CMR2','CMR3'])
    plt.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2),
          columnspacing=0.8,handletextpad=0.1)
    ax = plt.gca()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    plt.show()

In [ ]:
i = 0
A = GM[labelsMPC==1,:-1,i]
B = GM2[labelsMPC==1,:-1,i]
pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
print(pval)
reject_bh, pvals_corrected_bh, _, _ = multipletests(pval[1:], alpha=0.05, method='fdr_bh')
print(pvals_corrected_bh)

In [ ]:
for i in range(5):
    fig,ax = plt.subplots(2,1)
    A = GM[labelsMPC==2,:-1,i]
    B = GM[labelsMPC==1,:-1,i]
    ax[0].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['D1Nac'])
    ax[0].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='R-',c=ColorCode['D1Nac']+0.3)
    ax[0].set_xticks([])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:-1,i],axis=0)*0.1

    ax[0].plot(np.array([1,2,3])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')

    A = GM2[labelsMPC==2,:-1,i]
    B = GM2[labelsMPC==1,:-1,i]
    ax[1].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=ColorCode['D1Nac'],ls='--')
    ax[1].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=ColorCode['D1Nac']+0.3,ls='--')
    ax[1].set_xticks([1,2,3],['CMR1','CMR2','CMR3'])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:-1,i],axis=0)*0.1

    ax[1].plot(np.array([1,2,3])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')

    ax[0].set_ylabel('Pre-start',fontsize=24)
    ax[1].set_ylabel('Post-start',fontsize=24)
    if(i==0):
        x = -0.2
    else:
        x = -0.12
    fig.supylabel(FigLabs[i],fontsize=32,x=x)
    if i ==0:
        ax[0].legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.4),
          columnspacing=0.8,handletextpad=0.1)

In [ ]:
GM = np.zeros((len(CalcDatNAD1),3,5))
GM2 = np.zeros((len(CalcDatNAD1),3,5))
for i,C1 in enumerate(CalcDatNAD1):
    for j,C2 in enumerate(C1[1:-1]):
        T = TimesNAD1[i][j+1]
        CT = TimeCalcDatNAD1[i][j+1]
        CalcPre=C2[:,CT<T[0]]
        CalcPre = CalcPre[CalcPre.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPre]
        CalcPre = CalcPre[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPre)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM[i,j,:] = math.nan
            
        CalcPost=C2[:,CT>T[0]]
        CalcPost = CalcPost[CalcPost.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPost]
        CalcPost = CalcPost[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPost)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM2[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM2[i,j,:] = math.nan

In [ ]:
i = 4
pval = stats.ttest_ind(GM[:,:,i],GM2[:,:,i],nan_policy='omit').pvalue
print(pval)
reject_bh, pvals_corrected_bh, _, _ = multipletests(pval[1:], alpha=0.05, method='fdr_bh')
print(pvals_corrected_bh)

In [ ]:
GM[:,:-1,i].shape

In [ ]:
RomNum = ['i','ii','iii','iv','v']
for i in range(5):
    plt.errorbar([1,2,3],np.nanmean(GM[:,:,i],axis=0),stats.sem(GM[:,:,i],axis=0,nan_policy='omit'),
                 lw=3,label='Pre-start',c=ColorCode['D2Nac'])
    plt.errorbar([1,2,3],np.nanmean(GM2[:,:,i],axis=0),stats.sem(GM2[:,:,i],axis=0,nan_policy='omit'),
                 lw=3,c=ColorCode['D2Nac'],ls='--',
                 label='Post-start')
    pval_old = stats.ttest_ind(GM[:,:,i],GM2[:,:,i],nan_policy='omit').pvalue
    x = np.copy(pval_old)
    reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
    x[pval_old<0.05] = pval
    pval = np.copy(x)
    hval = np.max((np.nanmean(GM2[:,:,i],axis=0)+stats.sem(GM2[:,:,i],axis=0,nan_policy='omit'),
          np.nanmean(GM[:,:,i],axis=0)+stats.sem(GM[:,:,i],axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1
    m1 = (pval<0.05)*(pval>0.01)
    plt.plot(np.array([1,2,3])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    plt.plot(np.array([1,2,3])[m2]-0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m2]+0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    plt.plot(np.array([1,2,3])[m3]-0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m3]+0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.ylabel(FigLabs[i],fontsize=32)
    plt.xticks([1,2,3],['CMR1','CMR2','CMR3'])
    plt.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2),
          columnspacing=0.8,handletextpad=0.1)
    ax = plt.gca()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    plt.show()

In [ ]:
i = 3
A = GM[labelsNAD1==2,:,i]
B = GM[labelsNAD1==1,:,i]
pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
print(pval)
reject_bh, pvals_corrected_bh, _, _ = multipletests(pval[[0,-1]], alpha=0.05, method='fdr_bh')
print(pvals_corrected_bh)

In [ ]:
for i in range(5):
    fig,ax = plt.subplots(2,1)
    A = GM[labelsNAD1==2,:,i]
    B = GM[labelsNAD1==1,:,i]
    ax[0].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['D2Nac'])
    ax[0].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='R-',c=np.clip(ColorCode['D2Nac']+0.2,0,1))
    ax[0].set_xticks([])
    pval_old = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    try:
        x = np.copy(pval_old)
        reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
        x[pval_old<0.05] = pval
        pval = np.copy(x)
    except:
        pval = np.copy(pval_old)
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1

    m1 = (pval<0.05)*(pval>0.01)
    ax[0].plot(np.array([1,2,3])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    ax[0].plot(np.array([1,2,3])[m2]-0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax[0].plot(np.array([1,2,3])[m2]+0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    ax[0].plot(np.array([1,2,3])[m3]-0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax[0].plot(np.array([1,2,3])[m3]+0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    print(pval)
    A = GM2[labelsNAD1==2,:,i]
    B = GM2[labelsNAD1==1,:,i]
    ax[1].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=ColorCode['D2Nac'],ls='--')
    ax[1].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=np.clip(ColorCode['D2Nac']+0.2,0,1),ls='--')
    ax[1].set_xticks([1,2,3],['CMR1','CMR2','CMR3'])
    pval_old = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    try:
        x = np.copy(pval_old)
        reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
        x[pval_old<0.05] = pval
        pval = np.copy(x)
    except:
        pval = np.copy(pval_old)
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1

    m1 = (pval<0.05)*(pval>0.01)
    ax[1].plot(np.array([1,2,3])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    ax[1].plot(np.array([1,2,3])[m2]-0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax[1].plot(np.array([1,2,3])[m2]+0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    ax[1].plot(np.array([1,2,3])[m3]-0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax[1].plot(np.array([1,2,3])[m3]+0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    print(pval)
    print('_'*10)
    ax[0].set_ylabel('Pre-start',fontsize=24)
    ax[1].set_ylabel('Post-start',fontsize=24)
    if(i==0):
        x = -0.2
    else:
        x = -0.12
    fig.supylabel(FigLabs[i],fontsize=32,x=x)
    if i ==0:
        ax[0].legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.4),
          columnspacing=0.8,handletextpad=0.1)

In [ ]:
GM = np.zeros((len(CalcDatNAD2),3,5))
GM2 = np.zeros((len(CalcDatNAD2),3,5))
for i,C1 in enumerate(CalcDatNAD2):
    for j,C2 in enumerate(C1[1:-1]):
        T = TimesNAD2[i][j+1]
        CT = TimeCalcDatNAD2[i][j+1]
        CalcPre=C2[:,CT<T[0]]
        CalcPre = CalcPre[CalcPre.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPre]
        CalcPre = CalcPre[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPre)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM[i,j,:] = math.nan
            
        CalcPost=C2[:,CT>T[0]]
        CalcPost = CalcPost[CalcPost.sum(axis=1)>0]
        Spikes = [SpikeCount(c) for c in CalcPost]
        CalcPost = CalcPost[[len(S)>0 for S in Spikes]]
        M = np.corrcoef(CalcPost)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM2[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM2[i,j,:] = math.nan
        

In [ ]:
i = 4
pval = stats.ttest_ind(GM[:,:,i],GM2[:,:,i],nan_policy='omit').pvalue
print(pval)
reject_bh, pvals_corrected_bh, _, _ = multipletests(pval, alpha=0.05, method='fdr_bh')
print(pvals_corrected_bh)

In [ ]:
RomNum = ['i','ii','iii','iv','v']
for i in range(5):
    plt.errorbar([1,2,3],np.nanmean(GM[:,:,i],axis=0),stats.sem(GM[:,:,i],axis=0,nan_policy='omit'),
                 lw=3,label='Pre-start',c=ColorCode['mPFC'])
    plt.errorbar([1,2,3],np.nanmean(GM2[:,:,i],axis=0),stats.sem(GM2[:,:,i],axis=0,nan_policy='omit'),
                 lw=3,c=ColorCode['mPFC'],ls='--',
                 label='Post-start')
    pval_old = stats.ttest_ind(GM[:,:,i],GM2[:,:,i],nan_policy='omit').pvalue
    try:
        x = np.copy(pval_old)
        reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
        x[pval_old<0.05] = pval
        pval = np.copy(x)
    except:
        pval = np.copy(pval_old)
    hval = np.max((np.nanmean(GM2[:,:,i],axis=0)+stats.sem(GM2[:,:,i],axis=0,nan_policy='omit'),
          np.nanmean(GM[:,:,i],axis=0)+stats.sem(GM[:,:,i],axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1
    m1 = (pval<0.05)*(pval>0.01)
    plt.plot(np.array([1,2,3])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    plt.plot(np.array([1,2,3])[m2]-0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m2]+0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    plt.plot(np.array([1,2,3])[m3]-0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.plot(np.array([1,2,3])[m3]+0.09,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    plt.ylabel(FigLabs[i],fontsize=32)
    plt.xticks([1,2,3],['CMR1','CMR2','CMR3'])
    plt.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2),
          columnspacing=0.8,handletextpad=0.1)
    ax = plt.gca()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    plt.show()

In [ ]:
for i in range(5):
    fig,ax = plt.subplots(2,1)
    A = GM[labelsNAD2==2,:,i]
    B = GM[labelsNAD2==1,:,i]
    ax[0].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['mPFC'])
    ax[0].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='R-',c=np.clip(ColorCode['mPFC']+0.2,0,1))
    ax[0].set_xticks([])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    print(pval)
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1

    ax[0].plot(np.array([1,2,3])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')

    A = GM2[labelsNAD2==2,:,i]
    B = GM2[labelsNAD2==1,:,i]
    ax[1].errorbar([1,2,3],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=ColorCode['mPFC'],ls='--')
    ax[1].errorbar([1,2,3],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
                 lw=3,label='Pre-trial',c=np.clip(ColorCode['mPFC']+0.2,0,1),ls='--')
    ax[1].set_xticks([1,2,3],['CMR1','CMR2','CMR3'])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    hval = np.max((np.nanmean(A,axis=0)+stats.sem(A,axis=0,nan_policy='omit'),
          np.nanmean(B,axis=0)+stats.sem(B,axis=0,nan_policy='omit')),axis=0)+np.nanmean(GM[:,:,i],axis=0)*0.1

    ax[1].plot(np.array([1,2,3])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')

    ax[0].set_ylabel('Pre-start',fontsize=24)
    ax[1].set_ylabel('Post-start',fontsize=24)
    if(i==0):
        x = -0.2
    else:
        x = -0.12
    fig.supylabel(FigLabs[i],fontsize=32,x=x)
    if i ==0:
        ax[0].legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.4),
          columnspacing=0.8,handletextpad=0.1)